# Modelo de Cálculo

In [10]:
import warnings

class algoritmo_de_Markov:
    def __init__(self, reglas, alfabeto =None):
        if alfabeto is None:
            self.alfabeto = [elem for regla in reglas for elem in regla[:2]]
        else:
            self.alfabeto = set(alfabeto)
        self.reglas = reglas
        self._check_reglas()
        pass

    def _check_reglas(self):
        existe_terminal = False
        num_regla = 1
        for patron, remplazo, ind_terminal in self.reglas:
            for i in patron:
                if i not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{i}' no pertenece al alfabeto")
            for j in remplazo:
                if j not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{j}' no pertenece al alfabeto")
            
            if ind_terminal:
                existe_terminal = True
            
            num_regla += 1

        if not existe_terminal:
            warnings.warn("No hay reglas terminales")
        pass

    def _check_string(self, string):
        for i in string:
            if i not in self.alfabeto:
                raise ValueError(f"String: El símbolo '{i}' no pertenece al alfabeto")
        
    def exec_string(self, string, verbose=False):
        self._check_string(string)
        while True:
            aplicada = False  # Bandera para verificar si alguna regla fue aplicada
            for num_regla, (patron, remplazo, ind_terminal) in enumerate(self.reglas, start=1):
                pos = string.find(patron)
                if pos != -1:
                    string = string[:pos] + remplazo + string[pos + len(patron):]
                    aplicada = True
                    if verbose:
                        print(f"Estado string: {string}")
                        print(f'Se aplica la regla {num_regla}: "{patron}" -> "{remplazo}"')
                    if ind_terminal:
                        if verbose:
                            print("Regla terminal aplicada, deteniendo ejecución.")
                        return string  # Si es terminal, termina aquí
                    break  # Reinicia desde la primera regla
            if not aplicada:
                # Si no se aplicó ninguna regla en este ciclo, termina la ejecución
                if verbose:
                    print("No se ha podido aplicar ninguna regla más")
                return string

# Ejemplos Trabajo:

## 2: Patrones repetidos

In [11]:
alfabeto = ['a', 'b', 'X', 'Y']
R = [
    ("ab", "X", False),
    ("ba", "Y", True)
]

modelo = algoritmo_de_Markov(R, alfabeto)

print(modelo.alfabeto)
s= 'ababba'
print(s)
print(modelo.exec_string(s, verbose=True))


{'X', 'a', 'Y', 'b'}
ababba
Estado string: Xabba
Se aplica la regla 1: "ab" -> "X"
Estado string: XXba
Se aplica la regla 1: "ab" -> "X"
Estado string: XXY
Se aplica la regla 2: "ba" -> "Y"
Regla terminal aplicada, deteniendo ejecución.
XXY


## 3: Para indicar si una palabra está en un lenguaje

In [12]:
alfabeto = ['a', 'b', '', '0', '1']
R = [
    ('ba', '0', False),
    ('a0', '0', False),
    ('0a', '0', False),
    ('b0', '0', False),
    ('0b', '0', False),
    ('ab', '', False),
    ('a', '0', False),
    ('b', '0', False),
    ('0', '0', True),
    ('', '1', True)
]

pruebas = ['aaabbb', 'aabbb', 'aaa']

modelo = algoritmo_de_Markov(R, alfabeto)

for p in pruebas:
    print(f'\nProbando cadena: {p}')
    print('-'*40)
    if modelo.exec_string(p, verbose=True) == '1':
        print(f'La cadena {p} es valida')
    else:
        print(f'La cadena {p} no es valida')


Probando cadena: aaabbb
----------------------------------------
Estado string: aabb
Se aplica la regla 6: "ab" -> ""
Estado string: ab
Se aplica la regla 6: "ab" -> ""
Estado string: 
Se aplica la regla 6: "ab" -> ""
Estado string: 1
Se aplica la regla 10: "" -> "1"
Regla terminal aplicada, deteniendo ejecución.
La cadena aaabbb es valida

Probando cadena: aabbb
----------------------------------------
Estado string: abb
Se aplica la regla 6: "ab" -> ""
Estado string: b
Se aplica la regla 6: "ab" -> ""
Estado string: 0
Se aplica la regla 8: "b" -> "0"
Estado string: 0
Se aplica la regla 9: "0" -> "0"
Regla terminal aplicada, deteniendo ejecución.
La cadena aabbb no es valida

Probando cadena: aaa
----------------------------------------
Estado string: 0aa
Se aplica la regla 7: "a" -> "0"
Estado string: 0a
Se aplica la regla 3: "0a" -> "0"
Estado string: 0
Se aplica la regla 3: "0a" -> "0"
Estado string: 0
Se aplica la regla 9: "0" -> "0"
Regla terminal aplicada, deteniendo ejecución.

## 4: Corrección de textos

In [13]:
import string

alfabeto = list(string.ascii_lowercase)  # Genera ['a', 'b', ..., 'z']
alfabeto.extend(['ñ',' ', ',', '.', '?'])  # Agrega xtras

print(alfabeto)
R = [
    ('esdto', 'esto', False),
    ('qeu', 'que', False),
    ('hoal', 'hola', False),
    ('cmo', 'como', False),
    ('algo', '', True)
]


modelo = algoritmo_de_Markov(reglas=R, alfabeto=alfabeto)

string = 'hoal, cmo estas? esdto qeu te envio tiene errores.'
modelo.exec_string(string, verbose=True)


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ', ' ', ',', '.', '?']
Estado string: hoal, cmo estas? esto qeu te envio tiene errores.
Se aplica la regla 1: "esdto" -> "esto"
Estado string: hoal, cmo estas? esto que te envio tiene errores.
Se aplica la regla 2: "qeu" -> "que"
Estado string: hola, cmo estas? esto que te envio tiene errores.
Se aplica la regla 3: "hoal" -> "hola"
Estado string: hola, como estas? esto que te envio tiene errores.
Se aplica la regla 4: "cmo" -> "como"
No se ha podido aplicar ninguna regla más


'hola, como estas? esto que te envio tiene errores.'

## 5: Descifrando lenguajes cifrados

In [14]:
import string

reglas_morse = [
    ("/.-/", "A", False), ("/-.../", "B", False), ("/-.-./", "C", False), ("/-../", "D", False),
    ("/./", "E", False), ("/..-./", "F", False), ("/--./", "G", False), ("/..../", "H", False),
    ("/../", "I", False), ("/.---/", "J", False), ("/-.-/", "K", False), ("/.-../", "L", False),
    ("/--/", "M", False), ("/-./", "N", False), ("/---/", "O", False), ("/.--./", "P", False),
    ("/--.-/", "Q", False), ("/.-./", "R", False), ("/.../", "S", False), ("/-/", "T", False),
    ("/..-/", "U", False), ("/...-/", "V", False), ("/.--/", "W", False), ("/-..-/", "X", False),
    ("/-.--/", "Y", False), ("/--../", "Z", False),
    # Números
    ("/-----/", "0", False), ("/.----/", "1", False), ("/..---/", "2", False), ("/...--/", "3", False),
    ("/....-/", "4", False), ("/...../", "5", False), ("/-..../", "6", False), ("/--.../", "7", False),
    ("/---../", "8", False), ("/----./", "9", False),
    # Espacios entre letras y palabras en Morse
    ("_", " ", False)
]

reglas_morse_ordenadas = sorted(reglas_morse, key=lambda x: len(x[0]), reverse=True)

# Definicion alfabeto para representar morse
alfabeto_morse = ['.', '-', ' ', '/', '_']
alfabeto_morse.extend(string.ascii_uppercase)
alfabeto_morse.extend(string.digits)

# Crear una instancia del algoritmo de Markov para decodificar Morse
decodificador_morse = algoritmo_de_Markov(reglas_morse, alfabeto=alfabeto_morse)

# Decodificar un mensaje de ejemplo
mensaje_morse = "/.-/_/-.../_/-.-./"
resultado = decodificador_morse.exec_string(mensaje_morse, verbose=True)
print(f"Mensaje decodificado: {resultado}")


Estado string: A_/-.../_/-.-./
Se aplica la regla 1: "/.-/" -> "A"
Estado string: A_B_/-.-./
Se aplica la regla 2: "/-.../" -> "B"
Estado string: A_B_C
Se aplica la regla 3: "/-.-./" -> "C"
Estado string: A B_C
Se aplica la regla 37: "_" -> " "
Estado string: A B C
Se aplica la regla 37: "_" -> " "
No se ha podido aplicar ninguna regla más
Mensaje decodificado: A B C


C:\Users\bogdan.stefan\AppData\Local\Temp\ipykernel_2464\2936623201.py:30: UserWarning: No hay reglas terminales
  warnings.warn("No hay reglas terminales")
